In [3]:
# 实验2改用jupyter试试,觉得貌似这样也挺好用的
# 哇，jupyter代码写起来比python简单好多哦！
# 2022年6月10日15:19:49：

In [4]:
# 载入包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

## 1.载入数据部分

In [5]:
# 载入数据
path = 'ex2data1.txt'
data = pd.read_csv(path, header=None, names=['Exam 1', 'Exam 2', 'Admitted'])
data.head()

# add a ones column - this makes the matrix multiplication work out easier
data.insert(0, 'Ones', 1)

# set X (training data) and y (target variable)
cols = data.shape[1]
X = data.iloc[:,0:cols-1]
y = data.iloc[:,cols-1:cols]

# convert to numpy arrays and initalize the parameter array theta
X = np.array(X.values)
y = np.array(y.values)
theta = np.zeros(3)

检查一下当前的变量状况

In [6]:
print(theta)
X.shape, theta.shape, y.shape

[0. 0. 0.]


((100, 3), (3,), (100, 1))

## 下面计算损失和梯度

In [7]:
# 定义备份一些常用的变量
print(data)
print(data.shape)
m = data.shape[0]


    Ones     Exam 1     Exam 2  Admitted
0      1  34.623660  78.024693         0
1      1  30.286711  43.894998         0
2      1  35.847409  72.902198         0
3      1  60.182599  86.308552         1
4      1  79.032736  75.344376         1
..   ...        ...        ...       ...
95     1  83.489163  48.380286         1
96     1  42.261701  87.103851         1
97     1  99.315009  68.775409         1
98     1  55.340018  64.931938         1
99     1  74.775893  89.529813         1

[100 rows x 4 columns]
(100, 4)


In [8]:
# 定义sigmoid函数
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [9]:
# 测试一下sigmoid函数
# print(sigmoid(-4),sigmoid(0),sigmoid(4))

下面是代价函数：
$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}$

定义损失计算函数

In [10]:
# # 计算损失和梯度
# def costFunction(theta, X, y):

#     theta = np.matrix(theta)
#     X = np.matrix(X)
#     y = np.matrix(y)
#     m = len(X)

#     first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
#     second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))

#     J = np.sum(first - second) / m
#     grad = 1 / m * np.sum(    np.multiply((sigmoid(X * theta.T) - y), X)   , 0  )

#     return J, grad


def costFunction(theta, X, y):

    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    m = len(X)

    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))

    J = np.sum(first - second) / m
    # grad = 1 / m * np.sum(    np.multiply((sigmoid(X * theta.T) - y), X)   , 0  )

    return J


def gradient(theta, X, y):

    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    m = len(X)

    # J = np.sum(first - second) / m
    grad = 1 / m * np.sum(np.multiply((sigmoid(X * theta.T) - y), X), 0)

    return grad


测试一下求梯度，theta初始化为0时梯度为0.69

In [11]:
test_theta = [0,0,0]
cost = costFunction(test_theta, X, y)
grad = gradient(test_theta, X, y)
print('Expected cost (approx): 0.693\nExpected gradients (approx):-0.1000 -12.0092 -11.2628')
print('计算得到：', cost, grad)

Expected cost (approx): 0.693
Expected gradients (approx):-0.1000 -12.0092 -11.2628
计算得到： 0.6931471805599453 [[ -0.1        -12.00921659 -11.26284221]]


注意，我们实际上没有在这个函数中执行梯度下降，我们仅仅在计算一个梯度步长。在练习中，一个称为“fminunc”的Octave函数是用来优化函数来计算成本和梯度参数。由于我们使用Python，我们可以用SciPy的“optimize”命名空间来做同样的事情。

现在可以用SciPy's truncated newton（TNC）实现寻找最优参数。

In [12]:
import scipy.optimize as opt
result = opt.fmin_tnc(func=costFunction, x0=theta, fprime=gradient, args=(X, y))
print('计算得出最终的theta为:', result)

print('Expected theta (approx): -25.161 0.206 0.201')

计算得出最终的theta为: (array([-25.1613187 ,   0.20623159,   0.20147149]), 36, 0)
Expected theta (approx): -25.161 0.206 0.201


计算一下预测准确率

In [13]:
def predict(theta, X):
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.5 else 0 for x in probability]

In [15]:
theta_min = np.matrix(result[0])
predictions = predict(theta_min, X)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))

accuracy = 89%
